In [1]:
import numpy as np
import torch
from pkg_resources import packaging
import sys
sys.path.append("..")
from pathlib import Path
import clip.clip as clip
from PIL import Image
import os 
device = torch.device("cuda:7" if torch.cuda.is_available() else "cpu")

# model, preprocess = clip.load("ViT-B/32")
model, preprocess = clip.load(name="/data/sswang/NFT_search/models/ViT-L-14-336px.pt", device=device)
# 将模型加载到GPU中并切换到评估模式
# model.cuda(device).eval()
model.eval()
input_resolution = model.visual.input_resolution
context_length = model.context_length
vocab_size = model.vocab_size

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")
print("Input resolution:", input_resolution)
print("Context length:", context_length)
print("Vocab size:", vocab_size)

preprocess

/home/sswang/anaconda3/envs/img_matching/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Model parameters: 427,944,193
Input resolution: 336
Context length: 77
Vocab size: 49408


Compose(
    Resize(size=336, interpolation=bicubic, max_size=None, antialias=None)
    CenterCrop(size=(336, 336))
    <function _convert_image_to_rgb at 0x7efb11775550>
    ToTensor()
    Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))
)

np.stack(images) 函数将一个由多个图像组成的列表 images 沿着一个新轴（默认为 0）进行连接，生成一个新的数组。这个新的数组的维度比原来的数组多了一个维度，用于存储连接后的图像。如果 images 中的每个图像的尺寸都相同，那么连接后的数组的第一个维度将是 len(images)，第二个维度将是图像的高度，第三个维度将是图像的宽度，第四个维度将是图像的通道数。

这行代码是从一个图像列表中创建了一个 PyTorch 张量 image_input。np.stack(images) 函数沿着一个新轴（默认为 0）将图像列表进行连接。然后，torch.tensor() 将连接后的图像列表转换为 PyTorch 张量。.cuda() 方法将张量移动到 GPU 上进行加速运算。


`img_features` 是一个存放了100个图像的特征向量的列表。归一化操作 `img_features /= img_features.norm(dim=-1, keepdim=True)` 用于将特征向量进行归一化处理。

下面是对归一化操作的细节解释：

1. `img_features.norm(dim=-1, keepdim=True)`：这部分代码计算了 `img_features` 列表中每个特征向量的范数（即向量的长度）。`dim=-1` 表示在最后一个维度上进行计算，即对每个特征向量的元素进行平方求和，然后取平方根。

2. `keepdim=True`：这部分代码保持结果的维度与输入的 `img_features` 保持一致。这样，计算的结果将是一个具有相同形状的张量，每个特征向量的范数都被保留在一个单独的维度中。

3. `img_features /= img_features.norm(dim=-1, keepdim=True)`：这部分代码进行了归一化操作。通过除以每个特征向量的范数，将每个特征向量的长度缩放到1。`/=` 是就地除法运算符，它将结果保存回 `img_features` 列表中。

归一化操作的目的是将特征向量的尺度标准化，使其具有相似的大小。这有助于确保特征向量在计算相似度或进行其他操作时具有一致的权重和规模。通过将特征向量缩放到单位长度，可以使它们在相似度计算中具有更好的可比性和一致性。

In [88]:
import json

def check_dir(dir_path):
    """
    检查文件夹路径是否存在，不存在则创建

    Args:
        dir_path (str): 待检查的文件夹路径
    """
    if not os.path.exists(dir_path):
        try:
            os.makedirs(dir_path)
        except Exception as e:
            raise e

def load_json(json_path):
    """
    以只读的方式打开json文件

    Args:
        config_path: json文件路径

    Returns:
        A dictionary

    """
    with open(json_path, 'r', encoding='UTF-8') as f:
        return json.load(f)
    
def save_json(save_path, data):
    """
    Saves the data to a file with the given filename in the given path

    Args:
        :param save_path: The path to the folder where you want to save the file
        :param filename: The name of the file to save
        :param data: The data to be saved

    """
    with open(save_path, 'w', encoding='UTF-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

def is_str_Length_valid(str_list) -> bool:
    """
    判断字符串长度是否超过77

    Args:
        str_list (list): 字符串列表

    Returns:
        bool: 是否超过77
    """
    try:
        for str in str_list:
            clip.tokenize(str)
        return True
    except:
        return False


def tensorlize_imgs(model, img_path_list) -> torch.Tensor:
    """
    使用模型提取图片特征，返回图片特征向量列表

    Args:
        img_path_list (list): 图片路径列表

    Returns:
        torch.Tensor: 图片特征向量列表
    """

    images = []
    for img_path in img_path_list:

        image = Image.open(img_path).convert("RGB")
            # 首先将图片预处理成模型需要的格式
        images.append(preprocess(image))
        # 把图片加载进cuda中
    image_input = torch.tensor(np.stack(images)).cuda(device=device)
    with torch.no_grad():
        image_features = model.encode_image(image_input).float()
        # 将image_features从GPU移动到CPU，并返回
        return image_features.cpu()
            

def tensorlize_texts(model, text_tokens_list) -> torch.Tensor:
    """
    使用模型提取单句文本特征，返回文本特征向量列表

    Args:
        text_tokens_list (list): 文本列表

    Returns:
        torch.Tensor: 文本特征向量列表
    """
    text_tokens = clip.tokenize(text_tokens_list).cuda(device=device)
    with torch.no_grad():
        text_features = model.encode_text(text_tokens).float()
        # 将text_features从GPU移动到CPU，并返回
        # print(text_features.shape)
        return text_features.cpu().numpy().tolist()
    
def load_img_tensor(device, imgTensor_path):
    """
    加载图片的tensor 向量到指定cuda中

    Args:
        device (str): cuda
        img_path (str): 图片tensor路径

    Returns:
        torch.Tensor: 图片特征向量
    """
    # 加载json文件
    NFT_tensor_data = load_json(imgTensor_path)
    image_features = NFT_tensor_data['image_features']
    image_tensors = torch.tensor(image_features).to(device)
    return image_tensors

def load_des_tensor(device, desTensor_path):
    """
    加载描述的tensor 向量到指定cuda中

    Args:
        device (str): cuda
        img_path (str): 描述的tensor路径

    Returns:
        torch.Tensor: 图片特征向量
    """
    # 加载json文件
    NFT_tensor_data = load_json(desTensor_path)
    des_features = NFT_tensor_data['des_tensors'] 
    des_tensors = torch.tensor(des_features).to(device)
    # 将所有张量以第二个维度作为基准堆叠在一起
    des_tensors = torch.stack(tuple(des_tensors), dim=1)
    return des_tensors

def slide_window_tokenizer(text, window_size, step_size) -> list:
    """
    为了处理长度超过77的句子，这里设计滑动窗口分词

    Args:
        text (str): 将要被拆分的句子

    Returns:
        list: 拆分后的句子列表
    """
    words = text.split()
    sentences = []
    slide_window_list = [i for i in range(0, len(words) - 1, step_size) if i + step_size < len(words) - 1]
    for i in slide_window_list:
        sentence = ' '.join(words[i:i+window_size])
        sentences.append(sentence)
    return sentences

def calculate_cosine_similarity_topk(img_features, des_features, k = 10) -> tuple:
    """
    计算图片特征和描述特征的余弦相似度，并返回topk的结果

    Args:
        img_features (torch.tensor): 图像特征向量
        des_features (torch.tensor): 描述特征向量
        k (int, optional): 前k位结果. Defaults to 10.

    Returns:
        tuple: (topk的相似度，topk的索引)
    """
    # 对每一个特征向量进行归一化，使其范数为1
    img_features /= img_features.norm(dim=-1, keepdim=True)

    # 归一化描述特征
    des_features /= des_features.norm(dim=-1, keepdim=True)
    # similarity = des_features.cpu().numpy() @ img_features.cpu().numpy().T
    # 每个图像向量都与100个文本向量计算余弦相似度，然后计算100次
    text_probs = (100.0 * img_features @ des_features.T).softmax(dim=-1)
    top_probs, top_labels = text_probs.cpu().topk(k, dim=-1)
    return top_probs, top_labels


def slide_window_tokenizer(text, window_size, step_size) -> list:
    """
    为了处理长度超过77的句子，这里设计滑动窗口分词

    Args:
        text (str): 将要被拆分的句子

    Returns:
        list: 拆分后的句子列表
    """
    words = text.split()
    sentences = []
    slide_window_list = [i for i in range(0, len(words) - 1, step_size) if i + step_size < len(words) - 1]
    for i in slide_window_list:
        sentence = ' '.join(words[i:i+window_size])
        sentences.append(sentence)
    return sentences

def tensorlize_texts_slideWindow(model, text, window_size, step_size):
    """
    将长文本分块，然后对每块文本进行向量化，最后将这些向量平均。

    Args:
        model (CLIP): 使用的 CLIP 模型。
        text (str): 输入的文本。

    Returns:
        torch.Tensor: 文本特征向量。
    """

    chunks = slide_window_tokenizer(text, window_size, step_size)
    tensor_list = []
    for chunk in chunks:
        tokens = clip.tokenize([chunk]).cuda(device=device)
        with torch.no_grad():
            tensor_list.append(model.encode_text(tokens).float().cpu())
    # 使用所有块的平均值作为文本的表示
    avg_tensor = torch.mean(torch.stack(tensor_list), dim=0)
    return avg_tensor.numpy().tolist()


def tensorlize_valid_subsentence(model, text) -> torch.Tensor:
    """
    截取有效的子句，然后求特征向量值

    Args:
        model (CLIP): 使用的 CLIP 模型。
        text (str): 输入的文本。

    Returns:
        torch.Tensor: 文本特征向量。
    """
    text_tensor = None
    # 标记为False时，表示该句子无法被模型处理，需要进行拆分
    flag = False
    words = text.split()
    text_length = len(words)
    while not flag:
        try:
            text_tensor = tensorlize_texts(model, text)
            flag = True
        except:
            text_length -= 1
            text = ' '.join(words[:text_length])
    return text_tensor

def legalize_text(text) -> str:
    """
    截取有效长度的句子

    Args:
        text (str): 输入的文本。

    Returns:
        str: 截断之后的文本。
    """
    # 标记为False时，表示该句子无法被模型处理，需要进行拆分
    flag = False
    words = text.split()
    text_length = len(words)
    while not flag:
        try:
            clip.tokenize(text)
            flag = True
        except:
            text_length -= 1
            text = ' '.join(words[:text_length])
    return text

def handle_long_texts(model, text_list) -> list:
    """
    处理长文本，将长文本分块，然后对每块文本进行向量化，最后将这些向量平均。

    Args:
        model (CLIP): 使用的 CLIP 模型。
        text_list (list): 输入的文本列表。
        window_size (int): 窗口宽度
        step_size (int): 窗口移动的步长

    Returns:
        list: 文本特征向量列表。
    """
    # 先将长句子截断为有效短句子
    legal_text_list = list(map(legalize_text, text_list))
    text_features = tensorlize_texts(model, legal_text_list)
    return text_features


In [82]:
dataset_base_path = Path("/data/sswang/data/mini100")
target_dataset_path = Path("/data/sswang/data/mini100_tensor_V2")

In [83]:
import re
import copy

NFT_list_dict = load_json(dataset_base_path.joinpath("NFT_list.json"))

collection_list_copy = copy.deepcopy(NFT_list_dict["collection_list_copy"])

for key, value in collection_list_copy.items():
    NFT_collection_path = dataset_base_path.joinpath(value)
    print("开始处理：", NFT_collection_path.name, "...")
    check_dir(target_dataset_path.joinpath(NFT_collection_path.name))

    NFT_tensor_data = {}
    img_path_list = list(NFT_collection_path.joinpath("img").iterdir())
    # 提取图片名称
    img_name_list = [img_path.stem for img_path in img_path_list]

    # 提取图片特征向量
    image_features_CPU = tensorlize_imgs(model, img_path_list)

    # 提取文本特征向量
    des_tensor = []
    des_query_dict = load_json(NFT_collection_path.joinpath("description.json"))

    for img_name in img_name_list:
        # 去掉 description 中的序号（也就是1. 2. 3.）
        des_list = [re.sub(r'^\d+\. ', '', des) for des in des_query_dict[img_name]]
        des_features = None
        # 判断描述的长度有没有超过77
        if is_str_Length_valid(des_list) == False:
            # 如果超过77，就使用特殊处理方式
            des_features = handle_long_texts(model, des_list)
        else:
            des_features = tensorlize_texts(model, des_list)
        des_tensor.append(des_features)

    NFT_tensor_data["img_name_list"] = img_name_list
    NFT_tensor_data["image_features"] = image_features_CPU.numpy().tolist()
    NFT_tensor_data["des_tensors"] = des_tensor
    save_json(target_dataset_path.joinpath(NFT_collection_path.name, "NFT_tensor_data.json"), NFT_tensor_data)
    print("处理完成：", NFT_collection_path.name)
    
    # 将已经处理完成的项目从列表中删除
    del NFT_list_dict["collection_list_copy"][key]
    # 将处理完成的项目列表保存到json文件中
    save_json(dataset_base_path.joinpath("NFT_list.json"), NFT_list_dict)
    

开始处理： CryptoPunks ...
处理完成： CryptoPunks
开始处理： BoredApeYachtClub ...
处理完成： BoredApeYachtClub
开始处理： MutantApeYachtClub ...
处理完成： MutantApeYachtClub
开始处理： Azuki ...
处理完成： Azuki
开始处理： CLONEX ...
处理完成： CLONEX
开始处理： Moonbirds ...
处理完成： Moonbirds
开始处理： Doodles ...
处理完成： Doodles
开始处理： BoredApeKennelClub ...
处理完成： BoredApeKennelClub
开始处理： Meebits ...
处理完成： Meebits
开始处理： PudgyPenguins ...
处理完成： PudgyPenguins
开始处理： Cool Cats ...
处理完成： Cool Cats
开始处理： Beanz ...
处理完成： Beanz
开始处理： MechMinds ...
处理完成： MechMinds
开始处理： World of Women ...
处理完成： World of Women
开始处理： CrypToadz ...
处理完成： CrypToadz
开始处理： 0N1 Force ...
处理完成： 0N1 Force
开始处理： mfers ...
处理完成： mfers
开始处理： Karafuru ...
处理完成： Karafuru
开始处理： HAPE PRIME ...
处理完成： HAPE PRIME
开始处理： MekaVerse ...
处理完成： MekaVerse
开始处理： projectPXN ...
处理完成： projectPXN
开始处理： FLUF ...
处理完成： FLUF
开始处理： Hashmasks ...
处理完成： Hashmasks
开始处理： Moonbirds Oddities ...
处理完成： Moonbirds Oddities
开始处理： Creature World ...
处理完成： Creature World
开始处理： 3Landers ...
处理完成： 3Landers
开始处理： Phan

/home/sswang/anaconda3/envs/img_matching/lib/python3.9/site-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


处理完成： VeeFriends Series 2
开始处理： Lazy Lions ...
处理完成： Lazy Lions
开始处理： World of Women Galaxy ...
处理完成： World of Women Galaxy
开始处理： ALIENFRENS ...
处理完成： ALIENFRENS
开始处理： Prime Ape Planet ...
处理完成： Prime Ape Planet
开始处理： The Doge Pound ...
处理完成： The Doge Pound
开始处理： Sappy Seals ...
处理完成： Sappy Seals
开始处理： CyberKongz ...
处理完成： CyberKongz
开始处理： DigiDaigaku ...
处理完成： DigiDaigaku
开始处理： CoolmansUniverse ...
处理完成： CoolmansUniverse
开始处理： VOX Series 1 ...
处理完成： VOX Series 1
开始处理： Capsule ...
处理完成： Capsule
开始处理： Murakami.Flowers ...
处理完成： Murakami.Flowers
开始处理： SupDucks ...
处理完成： SupDucks
开始处理： Valhalla ...
处理完成： Valhalla
开始处理： DEGEN TOONZ ...
处理完成： DEGEN TOONZ
开始处理： Lives of Asuna ...
处理完成： Lives of Asuna
开始处理： Nakamigos ...
处理完成： Nakamigos
开始处理： Sneaky Vampire Syndicate ...
处理完成： Sneaky Vampire Syndicate
开始处理： Killer GF ...
处理完成： Killer GF
开始处理： Adam Bomb Squad ...
处理完成： Adam Bomb Squad
开始处理： Impostors Genesis ...
处理完成： Impostors Genesis
开始处理： CryptoSkulls ...
处理完成： CryptoSkulls
开始处理： MURI ...
处

In [ ]:
# 计算图像特征和描述特征的余弦相似度
image_features = load_img_tensor(device, target_dataset_path.joinpath("Prime Ape Planet", "NFT_tensor_data.json"))
des_features = load_des_tensor(device, target_dataset_path.joinpath("Prime Ape Planet", "NFT_tensor_data.json"))
des_features1, des_features2, des_feature3 = des_features
top_probs, top_labels = calculate_cosine_similarity_topk(image_features, des_features1, 10)
print(top_probs.shape)
print(top_probs)
print(top_labels.shape)
print(top_labels)


# collection 内部测试

In [131]:
import csv
dataset_base_path = Path("/data/sswang/NFT_search/data/CLIP")
# 定义表头文件
headers = ['collection', 'top1', 'top5', 'top10', 'top15', 'top20']
# 加载特征向量
collection_path_list = target_dataset_path.iterdir()
data = []
for collection in collection_path_list:
    # 找到路径下的tensor文件
    tensor_path = collection.joinpath("NFT_tensor_data.json")
    image_features = load_img_tensor(device, tensor_path)
    des_features = load_des_tensor(device, tensor_path)
    top_K = [1, 5, 10, 15, 20]
    print("开始计算：", collection.name, "...")
    data_item = [collection.name]
    for k in top_K:
        counter = 0
        for des_feature in des_features:
            _, top_labels = calculate_cosine_similarity_topk(image_features, des_feature, k)
            for index, top_index in enumerate(top_labels):
                if index in top_index:
                    counter += 1
        precision = round(counter / (3 * len(top_labels)) * 100, 2)
        data_item.append(precision)
        print("top", k, ":", precision, "%")
    print(data_item)
    data.append(data_item)
    print("计算完成：", collection.name)

# 将数据写入csv文件
with open(dataset_base_path.joinpath("precision.csv"), 'a+', newline='') as f:
    f_csv = csv.writer(f)
        # 写入表头
    f_csv.writerow(headers)
    # 写入数据
    f_csv.writerows(data)




开始计算： 0N1 Force ...
top 1 : 17.0 %
top 5 : 38.0 %
top 10 : 53.0 %
top 15 : 58.67 %
top 20 : 65.33 %
['0N1 Force', 17.0, 38.0, 53.0, 58.67, 65.33]
计算完成： 0N1 Force
开始计算： 3Landers ...
top 1 : 36.0 %
top 5 : 57.0 %
top 10 : 65.0 %
top 15 : 71.0 %
top 20 : 74.33 %
['3Landers', 36.0, 57.0, 65.0, 71.0, 74.33]
计算完成： 3Landers
开始计算： ALIENFRENS ...
top 1 : 30.33 %
top 5 : 55.0 %
top 10 : 69.33 %
top 15 : 77.33 %
top 20 : 82.33 %
['ALIENFRENS', 30.33, 55.0, 69.33, 77.33, 82.33]
计算完成： ALIENFRENS
开始计算： Acclimated​MoonCats ...
top 1 : 6.0 %
top 5 : 15.33 %
top 10 : 21.0 %
top 15 : 28.67 %
top 20 : 37.33 %
['Acclimated\u200bMoonCats', 6.0, 15.33, 21.0, 28.67, 37.33]
计算完成： Acclimated​MoonCats
开始计算： Adam Bomb Squad ...
top 1 : 12.67 %
top 5 : 26.67 %
top 10 : 34.0 %
top 15 : 40.0 %
top 20 : 46.0 %
['Adam Bomb Squad', 12.67, 26.67, 34.0, 40.0, 46.0]
计算完成： Adam Bomb Squad
开始计算： Akutars ...
top 1 : 21.67 %
top 5 : 42.33 %
top 10 : 52.67 %
top 15 : 58.33 %
top 20 : 62.67 %
['Akutars', 21.67, 42.33, 52.67, 5

# 所有的Collection放在一起测试


In [132]:
import csv
dataset_base_path = Path("/data/sswang/NFT_search/data/CLIP")
# 加载特征向量
collection_path_list = target_dataset_path.iterdir()
all_image_features = torch.tensor([]).to(device)
all_des_features = torch.tensor([]).to(device)
for collection in collection_path_list:
    # 找到路径下的tensor文件
    tensor_path = collection.joinpath("NFT_tensor_data.json")
    # 加载图像特征向量
    image_features = load_img_tensor(device, tensor_path)
    # 放入all_image_features中
    all_image_features = torch.cat((all_image_features, image_features), 0)
    # 加载文本特征向量
    des_features = load_des_tensor(device, tensor_path)
    # 放入all_des_features中
    all_des_features = torch.cat((all_des_features, des_features), 1)

print(all_image_features.shape)
print(all_des_features.shape)

data = ["all_collections"]

top_K = [1, 5, 10, 15, 20]
for k in top_K:
    counter = 0
    for des_feature in all_des_features:
        _, top_labels = calculate_cosine_similarity_topk(all_image_features, des_feature, k)
        for index, top_index in enumerate(top_labels):
            if index in top_index:
                counter += 1
    precision = round(counter / (3 * len(top_labels)) * 100, 2)
    data.append(precision)
    print("top", k, ":", precision, "%")
# 将数据写入csv文件
with open(dataset_base_path.joinpath("precision.csv"), 'a+', newline='') as f:
    f_csv = csv.writer(f)
    # 写入数据
    f_csv.writerow(data)




torch.Size([10000, 768])
torch.Size([3, 10000, 768])
top 1 : 21.52 %
top 5 : 38.15 %
top 10 : 45.71 %
top 15 : 50.04 %
top 20 : 53.32 %


In [120]:
all_image_features = torch.tensor([]).to(device)
all_des_features = torch.tensor([], device=device)
image_features_1 = torch.tensor([[0.1, 0.2, 0.3], [0.4, 0.5, 0.6]]).to(device)
print(image_features_1.shape)
image_features_2 = torch.tensor([[0.1, 0.2, 0.3], [0.4, 0.5, 0.6]]).to(device)
# 将图像特征向量放入all_image_features中
all_image_features = torch.cat((all_image_features, image_features_1), 0)
all_image_features = torch.cat((all_image_features, image_features_2), 0)
print(all_image_features.shape)
print(all_image_features)

print("\n______________________________________________________________________________________\n")

des_features_1 = torch.tensor([
                                [[0.1, 0.2, 0.3], [0.4, 0.5, 0.6]],
                                [[0.7, 0.8, 0.9], [0.10, 0.11, 0.12]],
                                [[0.13, 0.14, 0.15], [0.16, 0.17, 0.18]]
                                ]).to(device)
print(des_features_1.shape)
des_features_2 = torch.tensor([
                                [[0.1, 0.2, 0.3], [0.4, 0.5, 0.6]],
                                [[0.7, 0.8, 0.9], [0.10, 0.11, 0.12]],
                                [[0.13, 0.14, 0.15], [0.16, 0.17, 0.18]]
                                ]).to(device)

# 将文本特征向量放入all_des_features中
all_des_features = torch.cat((all_des_features, des_features_1), 1)
all_des_features = torch.cat((all_des_features, des_features_2), 1)

print(all_des_features.shape)
print(all_des_features)


torch.Size([2, 3])
torch.Size([4, 3])
tensor([[0.1000, 0.2000, 0.3000],
        [0.4000, 0.5000, 0.6000],
        [0.1000, 0.2000, 0.3000],
        [0.4000, 0.5000, 0.6000]], device='cuda:7')

______________________________________________________________________________________

torch.Size([3, 2, 3])
torch.Size([3, 4, 3])
tensor([[[0.1000, 0.2000, 0.3000],
         [0.4000, 0.5000, 0.6000],
         [0.1000, 0.2000, 0.3000],
         [0.4000, 0.5000, 0.6000]],

        [[0.7000, 0.8000, 0.9000],
         [0.1000, 0.1100, 0.1200],
         [0.7000, 0.8000, 0.9000],
         [0.1000, 0.1100, 0.1200]],

        [[0.1300, 0.1400, 0.1500],
         [0.1600, 0.1700, 0.1800],
         [0.1300, 0.1400, 0.1500],
         [0.1600, 0.1700, 0.1800]]], device='cuda:7')


In [129]:
# 定义表头
headers = ['collection', 'top1', 'top5', 'top10', 'top15', 'top20']
# 定义多行数据
data_rows = [['0N1 Force', 17.0, 38.0, 53.0, 58.67, 65.33], ['3Landers', 36.0, 57.0, 65.0, 71.0, 74.33], ['ALIENFRENS', 30.33, 55.0, 69.33, 77.33, 82.33], ['Acclimated\u200bMoonCats', 6.0, 15.33, 21.0, 28.67, 37.33], ['Adam Bomb Squad', 12.67, 26.67, 34.0, 40.0, 46.0], ['Akutars', 21.67, 42.33, 52.67, 58.33, 62.67], ['Anonymice', 4.33, 11.33, 17.33, 24.0, 29.0], ['Azuki', 51.33, 80.33, 88.67, 92.0, 94.0], ['Beanz', 41.33, 63.0, 69.0, 74.33, 77.67], ['Bears Deluxe', 15.67, 34.0, 49.33, 58.33, 63.33], ['BoredApeKennelClub', 26.0, 46.67, 56.33, 62.33, 67.33], ['BoredApeYachtClub', 27.67, 49.0, 60.67, 65.33, 68.0], ['Boss Beauties', 23.67, 46.67, 54.33, 59.33, 65.33], ['C-01 Official Collection', 0.67, 9.33, 15.33, 20.67, 26.0], ['CLONEX', 20.67, 48.67, 61.0, 69.67, 73.0], ['Capsule', 11.0, 25.33, 32.0, 39.0, 44.33], ['Chain Runners', 13.67, 38.0, 53.33, 61.33, 64.67], ['Chimpers', 10.0, 32.67, 44.0, 49.67, 54.0], ['Cool Cats', 48.33, 70.67, 76.67, 79.33, 81.33], ['CoolmansUniverse', 41.0, 63.0, 71.67, 77.67, 81.33], ['Creature World', 6.33, 23.33, 36.67, 44.33, 49.33], ['CrypToadz', 13.33, 30.0, 38.33, 45.67, 49.67], ['Crypto Bull Society', 10.33, 30.67, 40.0, 46.33, 50.0], ['CryptoBatz by Ozzy Osbourne', 15.33, 32.0, 40.67, 49.0, 54.67], ['CryptoMories', 36.0, 58.67, 69.0, 74.67, 75.67], ['CryptoPunks', 12.33, 29.33, 45.0, 59.33, 67.0], ['CryptoSkulls', 1.33, 4.0, 8.67, 14.33, 19.67], ['CyberKongz VX', 9.0, 19.33, 26.67, 30.67, 33.0], ['CyberKongz', 4.0, 18.0, 26.67, 37.0, 43.67], ['DEGEN TOONZ', 12.33, 38.33, 51.0, 59.67, 66.0], ['Deadfellaz', 13.33, 41.33, 57.33, 63.67, 71.0], ['DigiDaigaku', 22.67, 44.33, 54.0, 60.0, 64.33], ['Doodles', 38.0, 66.33, 73.33, 77.33, 80.0], ['FLUF', 24.67, 43.0, 51.0, 58.0, 61.33], ['Fighter', 18.67, 36.67, 43.67, 50.33, 54.67], ['ForgottenRunesWizardsCult', 14.33, 31.33, 41.33, 48.67, 53.33], ['Galactic Apes', 15.0, 34.0, 45.0, 56.67, 66.0], ['GalaxyEggs', 20.67, 39.33, 47.67, 53.0, 58.0], ['Genuine Undead', 11.33, 26.33, 38.0, 48.0, 52.33], ['Groupies', 19.33, 45.0, 57.67, 67.67, 74.33], ['HAPE PRIME', 29.67, 54.33, 61.33, 65.0, 71.67], ['Hashmasks', 21.0, 48.67, 59.0, 70.0, 74.67], ['Hero', 43.67, 70.0, 83.67, 89.0, 92.0], ['HypeBears', 63.67, 93.0, 96.67, 98.0, 98.67], ['Impostors Genesis', 47.0, 75.67, 85.0, 90.0, 93.0], ['Jungle Freaks', 76.33, 96.0, 97.33, 98.33, 99.33], ['KIA', 40.0, 70.67, 80.0, 87.67, 90.67], ['KILLABEARS', 52.33, 80.33, 87.67, 90.0, 92.67], ['KaijuKingz', 18.33, 40.33, 54.0, 61.33, 67.0], ['Kanpai Pandas', 13.0, 36.0, 44.0, 51.67, 58.67], ['Karafuru', 34.67, 68.33, 79.67, 87.33, 91.33], ['Keepers V2', 24.67, 55.0, 65.67, 73.67, 77.67], ['Killer GF', 58.33, 84.0, 92.67, 94.0, 94.67], ['Kiwami', 29.33, 57.33, 69.33, 77.67, 80.33], ['Lazy Lions', 51.0, 75.67, 86.67, 90.67, 92.33], ['Lil Heroes', 46.67, 74.33, 81.67, 87.0, 89.67], ['LilPudgys', 72.33, 82.33, 89.33, 91.67, 94.0], ['Lives of Asuna', 49.67, 75.67, 84.0, 88.0, 90.33], ['MOAR by Joan Cornella', 53.67, 78.33, 90.33, 93.67, 96.33], ['MURI', 39.67, 65.33, 78.67, 83.0, 87.33], ['MechMinds', 1.67, 5.67, 13.33, 19.0, 24.33], ['Meebits', 31.67, 59.67, 72.0, 80.67, 85.67], ['MekaVerse', 4.0, 18.67, 32.67, 45.33, 51.67], ['MetaHero', 20.67, 48.0, 63.0, 73.0, 78.67], ['Milady', 27.0, 63.0, 74.67, 83.33, 88.33], ['Moonbirds Oddities', 12.67, 35.33, 51.67, 59.0, 68.67], ['Moonbirds', 27.0, 50.0, 68.0, 74.0, 79.67], ['Murakami.Flowers', 14.33, 31.0, 42.0, 49.33, 55.67], ['MutantApeYachtClub', 17.67, 44.67, 57.67, 65.67, 71.0], ['MutantCats', 27.67, 50.67, 64.0, 73.67, 80.0], ['My Pet Hooligan', 32.33, 59.67, 70.67, 80.0, 84.0], ['Nakamigos', 51.67, 76.67, 85.67, 90.0, 93.33], ['OnChainMonkey', 1.33, 6.0, 12.0, 16.33, 23.0], ['Owls', 3.67, 9.33, 12.67, 20.67, 32.33], ['OxyaOriginProject', 12.33, 41.67, 62.0, 75.0, 79.0], ['Phanta Bear', 29.67, 52.0, 61.0, 64.67, 68.0], ['Prime Ape Planet', 45.33, 64.67, 73.67, 77.67, 78.67], ['PudgyPenguins', 65.67, 87.33, 93.0, 95.67, 98.0], ['Quirkies', 61.0, 86.0, 91.67, 93.67, 95.0], ['RumbleKongLeague', 46.33, 80.67, 87.67, 91.0, 93.33], ['SHIBOSHIS', 58.67, 78.67, 85.33, 91.33, 94.0], ['Sappy Seals', 43.0, 72.33, 80.0, 84.33, 88.67], ['Sevens Token', 14.33, 28.0, 34.67, 37.33, 38.67], ['Sneaky Vampire Syndicate', 45.67, 69.0, 81.33, 86.33, 90.67], ['SupDucks', 32.67, 59.67, 75.67, 83.33, 87.0], ['The Doge Pound', 50.33, 73.67, 86.67, 91.33, 95.0], ['The Heart Project', 46.0, 73.33, 82.67, 86.33, 90.33], ['The Humanoids', 30.67, 60.67, 71.67, 76.67, 81.0], ['VOX Series 1', 22.33, 49.0, 65.0, 73.67, 79.33], ['Valhalla', 69.0, 85.67, 92.67, 94.67, 96.0], ['VeeFriends Series 2', 45.33, 74.33, 88.67, 92.33, 95.67], ['Weirdo Ghost Gang', 52.0, 80.0, 88.0, 91.67, 94.67], ['WonderPals', 75.0, 92.33, 96.67, 98.0, 98.67], ['World of Women Galaxy', 15.0, 40.67, 50.67, 63.67, 69.67], ['World of Women', 39.67, 63.67, 75.67, 83.33, 86.67], ['a KID called BEAST', 13.67, 31.33, 43.33, 51.33, 59.67], ['inbetweeners', 50.0, 78.0, 91.33, 95.0, 97.33], ['mfers', 35.33, 71.67, 81.33, 86.0, 91.0], ['projectPXN', 20.0, 48.0, 64.33, 72.67, 77.67], ['tubby cats', 58.67, 82.0, 92.67, 95.67, 97.33]]

with open('output.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(headers)
    writer.writerows(data_rows)